Working Notebook to try things and donwload data from the quandl and tiingo

In [ ]:
import pandas_datareader.data as web
import pandas_datareader as pdr
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from empyrical import max_drawdown, annual_return, cagr, omega_ratio
from matplotlib.ticker import FixedLocator
from scipy.stats import geom
import math
import streamlit as st
from matplotlib.pyplot import rc
import datetime
import pandas_market_calendars as mcal

In [ ]:
symbol_oil = "EIA/PET_RWTC_D"
df_oil = web.DataReader(symbol_oil, 'quandl', '2000-01-01', '2021-02-12', api_key = quandl_key)
df_oil.to_csv("Oil_original.csv")
df_oil.reset_index().to_csv("Oil_flattened.csv")

In [ ]:
df_spy = pdr.get_data_tiingo('SPY', '2000-01-01', end='2021-02-12', api_key=tiingo_key)
df_spy.to_csv("SPY_original.csv")
df_spy.reset_index().to_csv("SPY_flattened.csv",index=False)

In [ ]:
df_gld = pdr.get_data_tiingo('GLD', '2000-01-01', end='2021-02-12', api_key=tiingo_key)
df_gld.to_csv("GLD_original.csv")
df_gld.reset_index().to_csv("GLD_flattened.csv",index=False)

In [ ]:
df_vglt = pdr.get_data_tiingo('VGLT', '2000-01-01', '2021-02-12', api_key=tiingo_key)
df_vglt.to_csv("VGLT_original.csv")
df_vglt.reset_index().to_csv("VGLT_flattened.csv")

In [ ]:
def read_data(file):
    data = pd.read_csv(file,parse_dates=['date'])
    data['date'] = data['date'].dt.tz_convert(None)
    data = data.set_index('date')
    return data


In [ ]:
def price_diff_loaded(dataset, startDate, endDate, dayprice = 'open'):
    '''
    Function takes in the dataset for a particular asset class, start and end dates, dayprice to use and calculates the absolute price change (in $)
    '''
    price_diff = dataset.loc[endDate,dayprice] - dataset.loc[startDate,dayprice]
    return price_diff

def price_ratio_loaded(dataset, startDate, endDate, dayprice = 'open'):
    '''
    Function takes in the dataset for a particular asset class, start and end dates, dayprice to use and calculates the relative price change
    '''
    price_ratio = dataset.loc[endDate,dayprice]/dataset.loc[startDate,dayprice]        
    return price_ratio

def dollar_cost_average_loaded(dataset, startDate, endDate, initial_investment = 0, regular_invest = 1000, freq = 10, dayprice = 'open'):
    '''
    Function takes in the dataset for a particular asset class, start and end dates, dayprice to use and calculates final value of the portfolio as a result of the dollar cost averaging
    '''
 
    dict_list = []
   
    invested = [] #tracks amount of invested capital up to this point
    portfolio_value = [] #track protfolio value
    nyse = mcal.get_calendar('NYSE')
    early = nyse.schedule(start_date=startDate, end_date=endDate)
    startDate = early.index[0]
    endDate = early.index[-1]
    dca_dates = list(early[::freq].index)
    #dca_dates = investment_dates_all = pd.bdate_range(start = startDate, end = endDate, freq = str(freq)+"B") #tracks the days when regular investment is made
    
    current_value = initial_investment*price_ratio_loaded(dataset, startDate, startDate, dayprice) 
    
    value_at_the_end = initial_investment*price_ratio_loaded(dataset, startDate, endDate, dayprice)
    invested = initial_investment
    portfolio_value = [current_value]
    dates = [startDate]
    previous_date = startDate
    
    history_dict = {'dates':startDate,'invested':initial_investment,'value_at_end':value_at_the_end,'current_value':current_value}
    dict_list.append(history_dict)
    
    for date in dca_dates[1:]:
        #add to the sum to track both current and at the end values of the portfolio
        current_value = current_value*price_ratio_loaded(dataset, startDate = previous_date, endDate = date, dayprice = dayprice) + regular_invest 
        value_at_the_end += price_ratio_loaded(dataset, startDate = date, endDate = endDate, dayprice = dayprice)*regular_invest 
        invested += regular_invest
        #log invested, portfolio_value
        tmp_dict = {'dates':date,'invested':invested,'value_at_end':value_at_the_end,'current_value':current_value}
        dict_list.append(tmp_dict)
        previous_date = date   
    current_value = current_value*price_ratio_loaded(dataset, startDate = date, endDate = endDate, dayprice = dayprice)
    tmp_dict = {'dates':endDate,'invested':invested,'value_at_end':value_at_the_end,'current_value':current_value}
    dict_list.append(tmp_dict)
    return pd.DataFrame(dict_list)  

   
def hold_overnight_ratio(dataset, startDate, endDate):
    '''
    This function calculates the difference assuming
    '''
    price_ratio_list = (dataset.loc[startDate:endDate,'open'][1:] - dataset.loc[startDate:endDate,'close'][:-1])/(dataset.loc[startDate:endDate,'close'][:-1]+0.000001)
    running_price_ratio = np.cumprod(1 + price_ratio_list)
    price_ratio = np.prod(1+price_ratio_list)
    return price_ratio, price_ratio_list, running_price_ratio 

def dollar_cost_average_hold_overnight(dataset, startDate, endDate, initial_investment = 0, regular_invest = 1000, freq = 10):
    '''
    Function takes in the dataset for a particular asset class, start and end dates, dayprice to use and calculates final value of the portfolio as a result of the dollar cost averaging
    ''' 
 
    dict_list = []

    nyse = mcal.get_calendar('NYSE')
    early = nyse.schedule(start_date=startDate, end_date=endDate)
    startDate = early.index[0]
    endDate = early.index[-1]
    dca_dates = list(early[::freq].index)
    #dca_dates = investment_dates_all = pd.bdate_range(start = startDate, end = endDate, freq = str(freq)+"B") #tracks the days when regular investment is made
    
    current_value = initial_investment*hold_overnight_ratio(dataset, startDate, startDate)[0] 
    
    value_at_the_end = initial_investment*hold_overnight_ratio(dataset, startDate, endDate)[0]
    invested = initial_investment
    previous_date = startDate
    
    history_dict = {'dates':startDate,'Invested':initial_investment,'Portfolio Value':current_value}
    dict_list.append(history_dict)
    
    for date in dca_dates[1:]:
        #add to the sum to track both current and at the end values of the portfolio
        current_value = current_value*hold_overnight_ratio(dataset, startDate = previous_date, endDate = date)[0] + regular_invest 
        invested += regular_invest
        #log invested, portfolio_value
        tmp_dict = {'dates':date,'Invested':invested,'Portfolio Value':current_value}
        dict_list.append(tmp_dict)
        previous_date = date   
    current_value = current_value*hold_overnight_ratio(dataset, startDate = date, endDate = endDate)[0]
    tmp_dict = {'dates':endDate,'Invested':invested,'Portfolio Value':current_value}
    dict_list.append(tmp_dict)
    return pd.DataFrame(dict_list).set_index('dates')


   
def dont_hold_overnight_ratio(dataset, startDate, endDate):
    '''
    This function calculates the difference assuming
    '''
    price_ratio_list = (dataset.loc[startDate:endDate,'close'] - dataset.loc[startDate:endDate,'open'])/(dataset.loc[startDate:endDate,'open']+0.000001)
    running_price_ratio = np.cumprod(1 + price_ratio_list)
    price_ratio = np.prod(1+price_ratio_list)
    return price_ratio, price_ratio_list, running_price_ratio 

def dollar_cost_average_dont_hold_overnight(dataset, startDate, endDate, initial_investment = 0, regular_invest = 1000, freq = 10):
    '''
    Function takes in the dataset for a particular asset class, start and end dates, dayprice to use and calculates final value of the portfolio as a result of the dollar cost averaging
    ''' 
 
    dict_list = []

    nyse = mcal.get_calendar('NYSE')
    early = nyse.schedule(start_date=startDate, end_date=endDate)
    startDate = early.index[0]
    endDate = early.index[-1]
    dca_dates = list(early[::freq].index)
    #dca_dates = investment_dates_all = pd.bdate_range(start = startDate, end = endDate, freq = str(freq)+"B") #tracks the days when regular investment is made
    
    current_value = initial_investment*dont_hold_overnight_ratio(dataset, startDate, startDate)[0] 
    
    value_at_the_end = initial_investment*dont_hold_overnight_ratio(dataset, startDate, endDate)[0]
    invested = initial_investment
    previous_date = startDate
    
    history_dict = {'dates':startDate,'Invested':initial_investment,'Portfolio Value':current_value}
    dict_list.append(history_dict)
    
    for date in dca_dates[1:]:
        #add to the sum to track both current and at the end values of the portfolio
        current_value = current_value*dont_hold_overnight_ratio(dataset, startDate = previous_date, endDate = date)[0] + regular_invest 
        invested += regular_invest
        #log invested, portfolio_value
        tmp_dict = {'dates':date,'Invested':invested,'Portfolio Value':current_value}
        dict_list.append(tmp_dict)
        previous_date = date   
    current_value = current_value*dont_hold_overnight_ratio(dataset, startDate = date, endDate = endDate)[0]
    tmp_dict = {'dates':endDate,'Invested':invested,'Portfolio Value':current_value}
    dict_list.append(tmp_dict)
    return pd.DataFrame(dict_list).set_index('dates')

In [ ]:
df_dont_hold_overnight = dollar_cost_average_dont_hold_overnight(df_spy, startDate=datetime.datetime(2018,7,10), endDate=datetime.datetime(2021,1,1))

In [ ]:
(df_dont_hold_overnight.index[-1] - df_dont_hold_overnight.index[0]).days

In [ ]:
df_hold_overnight = dollar_cost_average_hold_overnight(df_spy, startDate=datetime.datetime(2018,7,10), endDate=datetime.datetime(2021,1,1))

In [ ]:
df_hold_open = dollar_cost_average_loaded(df_spy, startDate='2018-07-05', endDate='2021-01-01')

In [ ]:
df_hold_close = dollar_cost_average_loaded(df_spy, startDate='2018-07-05', endDate='2021-01-01', dayprice = 'close')

In [ ]:
df_hold_open

In [ ]:
df_hold_open = df_hold_open.resample('12H')
    
# Get difference with array operations
difference = np.array(df_hold_open['current_value']) - np.array(df_hold_close['current_value'])
print(difference)
# Style and size
fig, ax = plt.subplots()
#sns.set_style("whitegrid")
fig.set_size_inches(15, 7)

# Plot Series
ax.fill_between(df_hold_open['dates'], y1=difference, y2=0, color='green', where=difference > 0, edgecolor='black')
ax.fill_between(df_hold_open['dates'], y1=difference, y2=0, color='red', where=difference < 0, edgecolor='black')
ax.plot(df_hold_open['dates'], difference, color='black', linewidth=.4)

ax.set_title('Lump Sum - DCA', size=18)
ax.set_ylabel('Current Value Difference($)', size=14)
ax.set_xlabel('Date of Investment', size=14)

plt.legend(['Amount','DCA','LumpSum'])
#plt.savefig('difference.png')
plt.show()